In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab\ Notebooks
![ ! -d "/content/drive/MyDrive/Colab Notebooks/COPM-Project/" ] && git clone https://github.com/egilltor17/COPM-Project.git
%cd /content/drive/MyDrive/Colab\ Notebooks/COPM-Project/
!git checkout MICCAI
!pip install -r requirements.txt

In [ ]:
# !watch nvidia-smi
!pip install wandb
import wandb
wandb.init()

In [ ]:
import os
import sys
sys.path.append(os.path.split(sys.path[0])[0])

from time import time
import numpy as np
import torch
import torch.backends.cudnn as cudnn
from torch.utils.data import DataLoader
from dataset.dataset import Dataset

from loss.Dice import DiceLoss
from loss.ELDice import ELDiceLoss
from loss.WBCE import WCELoss
from loss.Jaccard import JaccardLoss
from loss.SS import SSLoss
from loss.Tversky import TverskyLoss
from loss.Hybrid import HybridLoss
from loss.BCE import BCELoss

from net import net

import parameter as para

step_list = [0]
loss_plot = []

# 设置显卡相关
os.environ['CUDA_VISIBLE_DEVICES'] = para.gpu
cudnn.benchmark = para.cudnn_benchmark

# 定义网络
net = torch.nn.DataParallel(net).cuda()
net.train()

# 定义Dateset
train_ds = Dataset(para.training_set_path, para.training_set_path)

# 定义数据加载
# train_dl = DataLoader(dataset=train_ds, batch_size=para.batch_size, shuffle=True, num_workers=para.num_workers, pin_memory=para.pin_memory)
train_dl = DataLoader(dataset=train_ds, batch_size=1, shuffle=True, pin_memory=False)
print("Nr of training samples:", len(train_dl))

# 挑选损失函数
loss_func_list = [DiceLoss(), ELDiceLoss(), WCELoss(), JaccardLoss(), SSLoss(), TverskyLoss(), HybridLoss(), BCELoss()]
loss_func = loss_func_list[5]

# 定义优化器
opt = torch.optim.Adam(net.parameters(), lr=para.learning_rate)

# 学习率衰减
lr_decay = torch.optim.lr_scheduler.MultiStepLR(opt, para.learning_rate_decay)

# 深度监督衰减系数
alpha = para.alpha

In [ ]:
# 训练网络
print("Training epochs:", para.Epoch)
start = time()
for epoch in range(para.Epoch+1):
    mean_loss = []
    for step, (ct, seg) in enumerate(train_dl):
        ct = ct.cuda()
        seg = seg.cuda()

        opt.zero_grad()
        outputs = net(ct)
    
        loss1 = loss_func(outputs[0], seg)
        loss2 = loss_func(outputs[1], seg)
        loss3 = loss_func(outputs[2], seg)
        loss4 = loss_func(outputs[3], seg)
        loss = (loss1 + loss2 + loss3) * alpha + loss4
        mean_loss.append(loss4.item())

        # opt.zero_grad()
        loss.backward()
        opt.step()

        if step % 5 is 0:
            step_list.append(step_list[-1] + 1)
            loss_plot.append(loss4.item())

            print('epoch:{}, step:{}, loss1:{:.3f}, loss2:{:.3f}, loss3:{:.3f}, loss4:{:.3f}, time:{:.3f} min'
                  .format(epoch, step, loss1.item(), loss2.item(), loss3.item(), loss4.item(), (time() - start) / 60))
            
        
        # ct = ct.detach()
        # seg = seg.detach()


    # 保存模型
    if epoch % 50 is 0 and epoch is not 0:
        # 网络模型的命名方式为：epoch轮数+当前minibatch的loss+本轮epoch的平均loss
        torch.save(net.state_dict(), para.module_path + 'net{}-{:.3f}-{:.3f}.pth'.format(epoch, loss, sum(mean_loss) / len(mean_loss)))

    # 对深度监督系数进行衰减
    if epoch % 40 is 0 and epoch is not 0:
        alpha *= 0.8

    lr_decay.step()

# 深度监督的系数变化
  # 1.000
  # 0.800
  # 0.640
  # 0.512
  # 0.410
  # 0.328
  # 0.262
  # 0.210
  # 0.168
  # 0.134
  # 0.107
  # 0.086
  # 0.069
  # 0.055
  # 0.044
  # 0.035
  # 0.028
  # 0.023
  # 0.018
  # 0.014
  # 0.012
  # 0.009
  # 0.007
  # 0.006
  # 0.005
  # 0.004
  # 0.003
  # 0.002
  # 0.002
  # 0.002
  # 0.001
  # 0.001
  # 0.001
  # 0.001
  # 0.001
  # 0.000
  # 0.000

In [ ]:
import matplotlib.pyplot as plt

plt.plot(loss_plot)
plt.show()

In [ ]:
!pwd
!git status
!echo -e "[user]\n\tname = egilltor17\n\temail = egilltor17@ru.is" > ~/.gitconfig
!git add .
!git commit -m "Changes from Google Colab"
#!git pull origin MICCAI
!git push origin MICCAI